In [17]:
%load_ext autoreload
%autoreload 2

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import bottleneck as bn
from numba import jit

import warnings
from scipy.special import comb
from scipy.stats import circmean,circstd # circular statistics
from sklearn.utils import resample # for bootstrapping
from itertools import combinations
from datetime import datetime,timedelta
from nn_vorticity_module import least_square_method
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from haversine import haversine
import random
import matplotlib.dates as mdates
import multiprocessing as mp
from time import time
import scipy.linalg as la

warnings.simplefilter('ignore')
sns.set(style='whitegrid', context='poster', font_scale=1.2)
%matplotlib inline 

In [3]:
class Polygon:
    
    def __init__(self,i,comb,data):
        # initialize polygon
        #self.id = i
        #self.drifters=comb # indices of drifters in polygon
        self.lats=data.lat.values
        self.lons=data.lon.values
        self.com=np.array( [bn.nanmean(data.lon.values),bn.nanmean(data.lat.values)])
        #self.us=data.uv.real/100
        #self.vs=data.uv.imag/100
        self.length=[]
        self.aspect=[]
        self.angle=[]
    
    def p(self,i):
        # return coordinates of a point
        return [self.lons[i],self.lats[i]]
    
    def calc_lengths(self):
        lengths=[]
        ncc = len(self.lons)
        r = combinations(np.arange(ncc), 2) 
        
        k=0
        for i,j in r:
            lengths.append( haversine( self.p(i),self.p(j) ) )
            k+=1
            
        lengths=np.array(lengths)                   
        if np.sum(np.isfinite(lengths))==k:
            self.length = np.sqrt( np.mean(lengths**2) )
        else:
            self.length = np.nan           
    
    def least_square_method(self):
        #import gsw
        import scipy.linalg as la
        timeseries=True
        ncc = len(self.lons)
        dlon=[]
        dlat=[] 
        for i in range(ncc):
            # haversine(p1,p2)
            dlon.append(haversine( [self.lons[i],self.com[1]],self.com)*1000*np.sign(self.lons[i]-self.com[0]))
            dlat.append(haversine( [self.com[0],self.lats[i]],self.com)*1000*np.sign(self.lats[i]-self.com[1]))
        
        if not timeseries:
            R = np.mat( np.vstack( (np.ones((ncc,)) ,np.array(dlon), np.array(dlat) )).T )
            u0=np.mat(self.us).T
            v0=np.mat(self.vs).T

            A,_,_,_=la.lstsq(R,u0)
            B,_,_,_=la.lstsq(R,v0)
        
            self.A=A[1:]
            self.B=B[1:]

        points =np.vstack( [dlon,dlat] )
        if np.sum( np.isfinite(points))==2*npol:
            # careful with nans
            cov = np.cov(points)
            w,v = np.linalg.eig(cov)
            self.aspect = bn.nanmin(w)/bn.nanmax(w)
            
        else:
            self.aspect=np.nan
            #self.angle=np.nan

In [4]:
def makePolygons(i):
    criteria2 = data_chosen.particle.isin(combs[i])
    return Polygon(i,combs[i],data_chosen[criteria2])

def calc_properties(i):
    results[i].calc_lengths()
    results[i].least_square_method() 
    return results[i]

@jit
def find_percentiles(data):
    alpha = 0.75 # 95% confidence interval
    ordered = np.sort(data)
    lo = np.percentile(ordered,100*(1-alpha)/2,)
    hi = np.percentile(ordered,100*(alpha+(1-alpha)/2))
    return lo,hi

In [10]:
pwd

'/Users/sebastianessink/Dropbox (MIT)/deform/src/deformations'

In [12]:
data_path = '../../data/drifters/'
data = pd.read_pickle(data_path+'/posveldata_all.pkl')
N = data.particle.unique().size
npol=3
Nnpol = comb(N,npol,exact=True)
sampling_times = pd.date_range(data.index.unique()[0], periods=32, freq='1D')
T = len(sampling_times)
print('N=%d, npol=%d, Nnpol=%d, T=%d' %(N,npol,Nnpol,T))

combs=[]
for combi in combinations(np.arange(N),npol):
    combs.append(combi)

N=45, npol=3, Nnpol=14190, T=32


In [13]:
sampling_times

DatetimeIndex(['2015-09-03', '2015-09-04', '2015-09-05', '2015-09-06',
               '2015-09-07', '2015-09-08', '2015-09-09', '2015-09-10',
               '2015-09-11', '2015-09-12', '2015-09-13', '2015-09-14',
               '2015-09-15', '2015-09-16', '2015-09-17', '2015-09-18',
               '2015-09-19', '2015-09-20', '2015-09-21', '2015-09-22',
               '2015-09-23', '2015-09-24', '2015-09-25', '2015-09-26',
               '2015-09-27', '2015-09-28', '2015-09-29', '2015-09-30',
               '2015-10-01', '2015-10-02', '2015-10-03', '2015-10-04'],
              dtype='datetime64[ns]', freq='D')

In [16]:
results=[]

mean_length=np.zeros(T)
median_length=np.zeros(T)
std_length=np.zeros(T)
lo_length=np.zeros(T)
hi_length=np.zeros(T)

median_aspect=np.zeros(T)
mean_aspect=np.zeros(T)
lo_aspect=np.zeros(T)
hi_aspect=np.zeros(T)


start = time()
for t,tim in enumerate(sampling_times):
    
    results=[]
    if tim in data.index:
              
        # select subset in time
        data_chosen = data[data.index == tim]
        
        # make polygons
        results=[]
        pool = mp.Pool(8)     
        results = pool.map(makePolygons, range(Nnpol))
        pool.close()
        pool.join()
        
        # calculate properties
        pool = mp.Pool(8)     
        results = pool.map(calc_properties, range(Nnpol))
        pool.close()
        pool.join()
        
        # read geometry data
        asp = np.array([results[i].aspect for i in range(Nnpol)]).squeeze()
        ang = np.array([results[i].angle for i in range(Nnpol)]).squeeze()
        leng = np.array([results[i].length for i in range(Nnpol)]).squeeze()
        
        mean_length[t] = bn.nanmean(leng)
        std_length[t] = bn.nanstd(leng)
        median_length[t] = bn.nanmedian(leng)
        lo_length[t],hi_length[t] = find_percentiles(leng[np.isfinite(leng)])
        
        mean_aspect[t] = bn.nanmean(asp)
        median_aspect[t] = bn.nanmedian(asp)
        lo_aspect[t],hi_aspect[t] = find_percentiles(asp[np.isfinite(asp)])
        
        if np.mod(t,4)==0:
            print('1 step in %d %3.3f minutes' %(t,(time()-start)/60) )
        
    else:
        print('no data at that time.')
        
print('save data' )
df=[]
df = pd.DataFrame(index=sampling_times,data={'mean_length':mean_length,'median_length':median_length,'lo_length':lo_length,'hi_length':hi_length,
                                     'mean_aspect':mean_aspect,'median_aspect':median_aspect,'lo_aspect':lo_aspect,'hi_aspect':hi_aspect})

df['dtime']=df.index-df.index[0]
df['dtime']=df.dtime.dt.days.values
fname = 'deformation_n_6_T_90d_split_%d.pkl' %t
df.to_pickle(data_path+fname)
    
print('total %3.3f minutes' %((time()-start)/60) )

1 step in 0 0.049 minutes
1 step in 4 0.269 minutes
1 step in 8 0.518 minutes
1 step in 12 0.746 minutes
1 step in 16 0.975 minutes
1 step in 20 1.194 minutes
1 step in 24 1.423 minutes
1 step in 28 1.642 minutes
save data
total 1.799 minutes
